In [ ]:
import os
import sys
from pathlib import Path

# Models
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'model_path': '/models/tf2/my_ssd_mobilenet_v2/',
        'pipeline_file': 'pipeline.config'
    },
    'ssd_mobilenet_v2_fpn': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'model_path': '/models/tf2/my_ssd_mobilenet_v2_fpnlite/',
        'pipeline_file': 'pipeline.config'
    },
    'my_centernet_resnet50_v1_fpn': {
        'model_name': 'centernet_resnet50_v1_fpn_512x512_coco17_tpu-8',
        'model_path': '/models/tf2/my_centernet_resnet50_v1_fpn/',
        'pipeline_file': 'pipeline.config'
    },
    'my_centernet_resnet101_v1_fpn': {
        'model_name': 'centernet_resnet101_v1_fpn_512x512_coco17_tpu-8',
        'model_path': '/models/tf2/my_centernet_resnet101_v1_fpn/',
        'pipeline_file': 'pipeline.config'
    }
}

# Select a model to use.
selected_model = 'ssd_mobilenet_v2_fpn'

model_name = MODELS_CONFIG[selected_model]['model_name']
model_path = MODELS_CONFIG[selected_model]['model_path']
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Set Label Map (.pbtxt) path and pipeline.config path
label_map_pbtxt_fname = 'label-map.pbtxt'
pipeline_fname = pipeline_file

# Set .record path
test_record_fname = 'transformed/test.record'
train_record_fname = 'transformed/train.record'

# Set output directories and clean up
model_dir = 'training/'
output_dir = 'exported-models/'

#!rm -rf {model_dir} {output_dir}
os.makedirs(model_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Clone Tensorflow model repo
!git clone --quiet https://github.com/tensorflow/models.git

In [ ]:
# Compile protocol buffers
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
%cd ../..

In [ ]:
sys_path =  set(sys.path)
add_to_path = [ "models/research/slim", "models/research", "models"]
for path in add_to_path:
    p = str(Path(Path.cwd(),path))
    if p not in sys_path:
        sys.path.insert(0, p)
os.environ["PYTHONPATH"] = ";".join(sys.path)

In [ ]:
%cd models/research
!pip install .

!python object_detection/builders/model_builder_test.py
%cd ../..

In [ ]:
%cd models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = model_name + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
DEST_DIR = 'pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(model_name, DEST_DIR)

# Check downloaded files
#!echo {DEST_DIR}
#!ls -alh {DEST_DIR}

# Set fine tune checkpoint
fine_tune_checkpoint = os.path.join(DEST_DIR, "checkpoint/ckpt-0")
print("fine_tune_checkpoint: ", fine_tune_checkpoint)

%cd ../..

In [ ]:
!python xml_to_csv.py -i transformed/train/annotations -o train.csv
!python xml_to_csv.py -i transformed/test/annotations -o test.csv
!python xml_to_csv.py -i transformed/validate/annotations -o valdiate.csv

In [ ]:
!python generate_tfrecord.py -in train.csv -out train.tfrecord -images transformed/train/images
!python generate_tfrecord.py -in test.csv -out test.tfrecord -images transformed/test/images
!python generate_tfrecord.py -in valdiate.csv -out validate.tfrecord -images transformed/validate/images

## Tensorboard (?)

In [ ]:
LOG_DIR = model_dir
model_dir, output_dir

## Training

In [ ]:
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr

In [ ]:
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir} \
    --eval_timeout=60

### Exported

In [ ]:
!python models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path {pipeline_fname} \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_dir}